In [2]:
#Import required library
import tensorflow as tf
import numpy as np
import pathlib

#Get data
dataset_url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv'
data_dir = tf.keras.utils.get_file('daily-min-temperatures.csv', origin=dataset_url)
data_dir = pathlib.Path(data_dir)

#read data
time = []
temperature = []
with open(data_dir, 'r') as file:
    i = 0
    for line in file:
        if i == 0:
            i = i + 1
        else:
            time.append(line.split(',')[0])
            temperature.append(float(line.split(',')[1]))
#Declare Training parameter
split_size = 2500
window_size = 30
batch_size = 32
shuffle_buffer_size = 10000
#Split training and testing data
train_tempreature = temperature[:split_size]
test_tempreature = temperature[split_size:]
#Define function for data preprocessing
def windowed_dataset(series, window_size, batch_size, shuffle_buffer_size):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size+1, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda  window: window.batch(window_size+1))
    dataset = dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset
train_dataset = windowed_dataset(train_tempreature, window_size, batch_size, shuffle_buffer_size)
test_dataset = windowed_dataset(test_tempreature, window_size, batch_size, shuffle_buffer_size)
#Define model
model = tf.keras.models.Sequential([
    tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1), input_shape=[None]),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)
])
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001), loss='mse', metrics=['mse'])
history = model.fit(train_dataset, validation_data=test_dataset, epochs=30, verbose=0)
#Forecast
val_tempreature = temperature[split_size-window_size:]
def windowed_dataset_forcat(series, window_size, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda  window: window.batch(window_size))
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset
val_dataset = windowed_dataset_forcat(val_tempreature, window_size, 32)
result = np.array(model.predict(val_dataset))[0:-1]
forcast = []
for dat in result:
    forcast.append(dat[0])
print(tf.keras.metrics.mse(test_tempreature, forcast).numpy())

5.208928
